# MOVIE REVIEW CLASSIFICATION

Well, let cut it to the chase, this is a project to show case my understanding of IF-TDF and some basic idea of how processing text working in real world. With all the state-of-the-art machine learning model out-there I don't think mine would make any tiny change to the world. However admittedly, it's very valuable for me as someone who is trying to get my feet wet with machine learning world, and I would definitely recommend anyone who is learning attempt this tiny project ( To be honest, It would be more of a tutorial to me, I literally just get the code from our tutorial ...)

## Introduction
The World Wide Web contains an enourmous amount of documents that is easily accessiable. Obtaining the valuable insight from this corpus of information is considered to be next step of our industry evolution. We collect large amount feed back from customer and a tools to classify these feed back is in needs. Movie likewise is the most important information for user who go to movie. This small collab sessions will introduce some basic techniques that is used in text classification: Preprocessing, TF-IDF, Logistic Regression.


#Loading Data

The files contains list of reviews and its sentiments. First we need to load it to a pandas data frame, and check for data type and columns.

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
review = pd.read_csv('/content/drive/My Drive/Student Files/FTMLE - Tonga/Data/movie_review.csv', encoding='utf-8', sep='\t')


In [25]:
review.head()

,id,review,sentiment
0,5814_8,With all this stuff going down at the moment w...,1
1,2381_9,"\The Classic War of the Worlds\"" by Timothy Hi...",1
2,7759_3,The film starts with a manager (Nicholas Bell)...,0
3,3630_4,It must be assumed that those who praised this...,0
4,9495_8,Superbly trashy and wondrously unpretentious 8...,1


In [26]:
review.tail()

,id,review,sentiment
22495,3453_3,It seems like more consideration has gone into...,0
22496,5064_1,I don't believe they made this film. Completel...,0
22497,10905_3,"Guy is a loser. Can't get girls, needs to buil...",0
22498,10194_3,This 30 minute documentary BuÃ±uel made in the...,0
22499,8478_8,I saw this movie as a child and it broke my he...,1


##Preprocessing
Raw data can be hard for machine learning algorithem process, sometime, impossible. To improve training time and accuracy we can apply some processing ( called preprocessing ) on our data.

## Stemming
The words can have numerous forms, even thought they have roughly same nuance in the text, but the machine learning model treat them differently, to optimize training speed of the model, it's may be of benefit to reduce all the word to its stem.

In [27]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
porter = PorterStemmer()
import re
def tokenizer_porter(text):
    # Stemming
    return [porter.stem(word) for word in text.split()]

def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text
# Create a tfidf vectorizer
tfidf = TfidfVectorizer(stop_words=stop, tokenizer = tokenizer_porter,preprocessor = preprocessor)

#Start Traing
One of the most simple machine learning model to classify is logistic regression, and we are going to use that model in this collab.

In [0]:
from sklearn.model_selection import train_test_split
X = review.review
y = review.sentiment
X_train,x_test,y_train,y_test = train_test_split(X, y, test_size = .2)

We create a Pipeline and start feeding the data in to the pipeline.

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
clf = Pipeline([('vect',tfidf),('clf',LogisticRegression(random_state=0))])
clf.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x7f4714be2ea0>,
                                 smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', '...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x7f4714be2f28>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
         

Checking the model accuracy.

In [32]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
predictions = clf.predict(x_test)
accuracy_score(y_test,predictions)

0.8855555555555555

# Retest on validation data


In [0]:
retest = pd.read_csv('/content/drive/My Drive/Student Files/FTMLE - Tonga/Data/movie_review_evaluation.csv', encoding='utf-8', sep='\t')


In [35]:
retest.head()

,id,review
0,10633_1,I watched this video at a friend's house. I'm ...
1,4489_1,`The Matrix' was an exciting summer blockbuste...
2,3304_10,This movie is one among the very few Indian mo...
3,3350_3,The script for this movie was probably found i...
4,1119_1,Even if this film was allegedly a joke in resp...


In [0]:
X_validation = retest['review']

y_validation = clf.predict(X_validation)
retest['prediction'] = y_validation

In [0]:
retest.to_csv('drive/My Drive/vuquocan.csv')